In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
df_application_train = pd.read_csv('application_train.csv',header=0,error_bad_lines=False,index_col=['SK_ID_CURR'],nrows=100000)
df_previous_application = pd.read_csv('previous_application.csv',header=0,error_bad_lines=False,index_col=['SK_ID_CURR'],nrows=500000)
df_installment_payments = pd.read_csv('installments_payments.csv',header=0,error_bad_lines=False,index_col=['SK_ID_CURR'],nrows=500000)
df_bureau = pd.read_csv('bureau.csv',header=0,error_bad_lines=False,index_col=['SK_ID_CURR'],nrows=500000)
df_bureau_bal = pd.read_csv('bureau_balance.csv',header=0,error_bad_lines=False,index_col=0,nrows=5000)
df_pos_cash_bal = pd.read_csv('POS_CASH_balance.csv',header=0,error_bad_lines=False,index_col=['SK_ID_CURR'],nrows=5000)
df_credit_card_bal = pd.read_csv('credit_card_balance.csv',header=0,error_bad_lines=False,nrows=5000)

dflist = [df_application_train,df_previous_application,df_installment_payments,df_bureau,df_bureau_bal,df_pos_cash_bal,df_credit_card_bal]


Started with Previous Application Dataset because it had historical information about the application. My main focus was on 3 noticable columns. 

1. FLAG_LAST_APPL_PER_CONTRACT: If this column had the value of 'N', then it was more than likely a mistake or duplicate of another application. My thought is that this maybe unnecessary data to the purpose of this project.

2. AMT_APPLICATION: This column states how much the applicant has asked for from the bank. 

3. AMT_CREDIT: This column states how much the bank as credited the applicant.

        My thought is if (2) and (3) are 0, then the application is incomplete and it would not provide me with additional information about the application. Thus I should remove these rows from my dataset.
        
        
In the next cell, I have worked on removing those rows from the previous_application dataset. 

In [3]:
df_previous_application = df_previous_application.drop(df_previous_application[df_previous_application['FLAG_LAST_APPL_PER_CONTRACT']=='N'].index)
df_previous_application = df_previous_application.drop(df_previous_application[(df_previous_application['AMT_APPLICATION']==0.0) & (df_previous_application['AMT_CREDIT']==0.0)].index)



In the next cell, the 'RATE_DOWN_PAYMENT','RATE_INTEREST_PRIMARY', and 'RATE_INTEREST_PRIVILEGED' columns had 'NaN' values. Because I will be aggregating the columns to merge them into the main dataset (application_train), I have replaced the NaN values with 0. 

In [4]:
df_previous_application[['RATE_DOWN_PAYMENT','RATE_INTEREST_PRIMARY','RATE_INTEREST_PRIVILEGED']] = df_previous_application[['RATE_DOWN_PAYMENT','RATE_INTEREST_PRIMARY','RATE_INTEREST_PRIVILEGED']].fillna(0.0)

Previous application dataset has all of the previous application available. Currently, it has a 1:many relationship to the application dataset, so I will need to aggregate the columns before merging. 

In [5]:
df_previous_application.head()

,SK_ID_PREV,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
271877,2030495,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
108129,2802425,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
122040,2523466,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
176158,2819243,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
199383,1383531,Cash loans,23703.930,315000.0,340573.5,NaN,315000.0,SATURDAY,8,Y,...,XNA,18.0,low_normal,Cash X-Sell: low,365243.0,-654.0,-144.0,-144.0,-137.0,1.0


Below is where I aggreagated the previous_application dataset by using index 'SK_ID_CURR'. The purpose of this is so I can provide a 1:1 relationship to the main

In [6]:
df_prevapp_pivot = pd.pivot_table(df_previous_application,values=df_previous_application.columns,index='SK_ID_CURR',
                                  aggfunc={'SK_ID_PREV':(lambda x: len(x.unique())), 'NAME_CONTRACT_TYPE': (lambda x: len(x.unique())), 
                                           'AMT_ANNUITY':'sum', 'AMT_APPLICATION': 'sum','AMT_CREDIT':'sum', 'AMT_DOWN_PAYMENT':'sum', 
                                           'AMT_GOODS_PRICE':'sum','WEEKDAY_APPR_PROCESS_START':(lambda x: len(x.unique())), 
                                           'HOUR_APPR_PROCESS_START':np.mean,'FLAG_LAST_APPL_PER_CONTRACT':(lambda x: len(x.unique())), 
                                           'NFLAG_LAST_APPL_IN_DAY':(lambda x: len(x.unique())),'RATE_DOWN_PAYMENT':np.mean, 
                                           'RATE_INTEREST_PRIMARY':np.mean,'RATE_INTEREST_PRIVILEGED':np.mean, 
                                           'NAME_CASH_LOAN_PURPOSE':(lambda x: len(x.unique())),
                                           'NAME_CONTRACT_STATUS':(lambda x:x.map({'Approved':1,'Canceled':0,'Refused':0,'Unused offer':1}).mean()), 
                                           'DAYS_DECISION':np.mean, 'NAME_PAYMENT_TYPE':(lambda x: len(x.unique())),
                                           'CODE_REJECT_REASON':(lambda x: len(x.unique())),'NAME_TYPE_SUITE':(lambda x: len(x.unique())), 
                                           'NAME_CLIENT_TYPE':(lambda x: len(x.unique())),'NAME_GOODS_CATEGORY':(lambda x: len(x.unique())), 
                                           'NAME_PORTFOLIO':(lambda x: len(x.unique())), 'NAME_PRODUCT_TYPE':(lambda x: len(x.unique())),
                                           'CHANNEL_TYPE':(lambda x: len(x.unique())), 'SELLERPLACE_AREA':(lambda x: len(x.unique())), 
                                           'NAME_SELLER_INDUSTRY':(lambda x: len(x.unique())),'CNT_PAYMENT':'sum', 
                                           'NAME_YIELD_GROUP':(lambda x: len(x.unique())), 'PRODUCT_COMBINATION':(lambda x: len(x.unique())),
                                           'DAYS_FIRST_DRAWING':'sum', 'DAYS_FIRST_DUE':'sum', 'DAYS_LAST_DUE_1ST_VERSION':'sum',
                                           'DAYS_LAST_DUE':'sum', 'DAYS_TERMINATION':'sum', 'NFLAG_INSURED_ON_APPROVAL':np.mean})

In [7]:
df_prevapp_pivot.head()

,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,CHANNEL_TYPE,CNT_PAYMENT,CODE_REJECT_REASON,DAYS_DECISION,DAYS_FIRST_DRAWING,...,NAME_YIELD_GROUP,NFLAG_INSURED_ON_APPROVAL,NFLAG_LAST_APPL_IN_DAY,PRODUCT_COMBINATION,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,SELLERPLACE_AREA,SK_ID_PREV,WEEKDAY_APPR_PROCESS_START
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3951.000,24835.5,23787.0,2520.0,24835.5,1,8.0,1,-1740.0,365243.0,...,1,0.0,1,1,0.104326,0.0,0.0,1,1,1
100006,24246.000,675000.0,675000.0,0.0,675000.0,1,48.0,1,-181.0,365243.0,...,1,0.0,1,1,0.000000,0.0,0.0,1,1,1
100007,29520.585,405000.0,464400.0,0.0,405000.0,2,66.0,1,-866.0,365243.0,...,2,0.0,1,2,0.000000,0.0,0.0,2,2,2
100008,17885.835,162598.5,162598.5,0.0,162598.5,1,10.0,1,-370.0,365243.0,...,1,0.0,1,1,0.000000,0.0,0.0,1,1,1
100009,8996.760,98239.5,98239.5,0.0,98239.5,1,12.0,1,-449.0,365243.0,...,1,0.0,1,1,0.000000,0.0,0.0,1,1,1


# Merging previous_application dataset into application_train dataset 

In [8]:


df = df_application_train.merge(df_prevapp_pivot,on =['SK_ID_CURR'],suffixes=('_app_train','_prev_app'),how='left')

# Note the odd data within the columns

Days_Employed: 365243 default value



In [9]:
print(df.describe())

              TARGET   CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT_app_train  \
count  100000.000000  100000.000000      1.000000e+05          1.000000e+05   
mean        0.080930       0.417390      1.694261e+05          5.990034e+05   
std         0.272729       0.721014      3.835007e+05          4.020520e+05   
min         0.000000       0.000000      2.565000e+04          4.500000e+04   
25%         0.000000       0.000000      1.125000e+05          2.700000e+05   
50%         0.000000       0.000000      1.440000e+05          5.130405e+05   
75%         0.000000       1.000000      2.025000e+05          8.086500e+05   
max         1.000000      12.000000      1.170000e+08          4.050000e+06   

       AMT_ANNUITY_app_train  AMT_GOODS_PRICE_app_train  \
count           99993.000000               9.991900e+04   
mean            27085.325928               5.383451e+05   
std             14459.617534               3.693977e+05   
min              1980.000000               4.500000e

# Aggreagated Bureau & Installment datasets and then merged into dataset

In [10]:
df_bureau_pivot = pd.pivot_table(df_bureau,values=df_bureau.columns,index='SK_ID_CURR',
                                 aggfunc={'SK_ID_BUREAU':'count','CREDIT_ACTIVE':'count',
                                          'CREDIT_CURRENCY':(lambda x: len(x.unique())),'DAYS_CREDIT':np.min,
                                          'CREDIT_DAY_OVERDUE':'sum','DAYS_CREDIT_ENDDATE':'sum', 
                                          'DAYS_ENDDATE_FACT':'sum', 'AMT_CREDIT_MAX_OVERDUE':'sum', 
                                          'CNT_CREDIT_PROLONG':'sum', 'AMT_CREDIT_SUM':'sum','AMT_CREDIT_SUM_DEBT':'sum', 
                                          'AMT_CREDIT_SUM_LIMIT':'sum', 'AMT_CREDIT_SUM_OVERDUE':'sum',
                                          'CREDIT_TYPE':(lambda x: len(x.unique())), 'DAYS_CREDIT_UPDATE':np.min, 
                                          'AMT_ANNUITY':'sum'})

In [11]:
df = df.merge(df_bureau_pivot,on =['SK_ID_CURR'],suffixes=('_curr','_bur'),how='left')

In [12]:
df_installment_payments_pivot = pd.pivot_table(df_installment_payments,values=df_installment_payments.columns,index='SK_ID_CURR',aggfunc={'SK_ID_PREV':(lambda x: len(x.unique())),'NUM_INSTALMENT_VERSION':'sum', 'NUM_INSTALMENT_NUMBER':'sum','DAYS_INSTALMENT':'sum', 'DAYS_ENTRY_PAYMENT':'sum', 'AMT_INSTALMENT':'sum','AMT_PAYMENT':'sum'})                                                                                                                      

# Added Columns to show differences before merging


AMT_INSTAL_PAY_DIFF: I believe that knowing how much the applicant has paid back in the past would be a good indicator for the future

NUM_INSTAL_VERSION_NUM_DIFF: I believe knowing how fast or slow the previous load was paid would also be a good indicator.

DAY_INSTAL_ENTRY_DIFF: Similar to NUM_INSTAL_VERSION_NUM_DIFF

In [13]:
df_installment_payments_pivot['AMT_INSTAL_PAY_DIFF'] = df_installment_payments_pivot['AMT_INSTALMENT']-df_installment_payments_pivot['AMT_PAYMENT']
df_installment_payments_pivot['NUM_INSTAL_VERSION_NUM_DIFF'] = df_installment_payments_pivot['NUM_INSTALMENT_VERSION']-df_installment_payments_pivot['NUM_INSTALMENT_NUMBER']
df_installment_payments_pivot['DAY_INSTAL_ENTRY_DIFF'] = df_installment_payments_pivot['DAYS_INSTALMENT']-df_installment_payments_pivot['DAYS_ENTRY_PAYMENT']


In [14]:
df = df.merge(df_installment_payments_pivot,on =['SK_ID_CURR'],suffixes=('_curr2','_instpay'),how='left')

In [15]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 100002 to 216090
Columns: 183 entries, TARGET to DAY_INSTAL_ENTRY_DIFF
dtypes: float64(127), int64(40), object(16)
memory usage: 140.4+ MB
None


In [16]:
df['BANKED']= df.index.isin(df_bureau.index)

To find the outliers and NaNs, I went through each column using the code below and switched only the column index value of the iloc function. Through this method, I was able to drop about 83 columns because it did not have enough values to fill, did not seem to have any factor to the business case (e.g. weekday of application), or had low variety of values (e.g. 99% of values in the given column are True). 

As a result, I decreased the size of the dataframe from 140.4MB to 84MB as seen below.  



In [17]:
df.iloc[:,183].value_counts(dropna=False)

False    67568
True     32432
Name: BANKED, dtype: int64

In [18]:
df = df.drop(columns=['NAME_TYPE_SUITE_app_train','FLAG_MOBIL',
                'FLAG_EMP_PHONE','FLAG_PHONE','FLAG_WORK_PHONE',
                'FLAG_EMAIL','FLAG_CONT_MOBILE','FLAG_PHONE',
                'WEEKDAY_APPR_PROCESS_START_app_train',
                'HOUR_APPR_PROCESS_START_app_train','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3',
                'APARTMENTS_AVG','BASEMENTAREA_AVG','YEARS_BEGINEXPLUATATION_AVG',
               'YEARS_BUILD_AVG','COMMONAREA_AVG','ELEVATORS_AVG',
               'ENTRANCES_AVG','FLOORSMAX_AVG','FLOORSMIN_AVG',
                'LANDAREA_AVG','LIVINGAPARTMENTS_AVG','LIVINGAREA_AVG',
               'NONLIVINGAPARTMENTS_AVG','NONLIVINGAREA_AVG','APARTMENTS_MODE',
                'BASEMENTAREA_MODE','YEARS_BEGINEXPLUATATION_MODE',
               'YEARS_BUILD_MODE','COMMONAREA_MODE','ELEVATORS_MODE',
               'ENTRANCES_MODE','FLOORSMAX_MODE','FLOORSMIN_MODE','LANDAREA_MODE',
                'LIVINGAPARTMENTS_MODE','LIVINGAREA_MODE','NONLIVINGAPARTMENTS_MODE',
               'NONLIVINGAREA_MODE','APARTMENTS_MEDI','BASEMENTAREA_MEDI',
                'YEARS_BEGINEXPLUATATION_MEDI','YEARS_BUILD_MEDI',
               'COMMONAREA_MEDI','ELEVATORS_MEDI','ENTRANCES_MEDI',
                'FLOORSMAX_MEDI','FLOORSMIN_MEDI','LANDAREA_MEDI',
                'LIVINGAPARTMENTS_MEDI','LIVINGAREA_MEDI',
                'NONLIVINGAPARTMENTS_MEDI','NONLIVINGAREA_MEDI',
                'FONDKAPREMONT_MODE','HOUSETYPE_MODE',
                'TOTALAREA_MODE','WALLSMATERIAL_MODE',
                'EMERGENCYSTATE_MODE','DAYS_FIRST_DRAWING',
               'DAYS_FIRST_DUE','DAYS_LAST_DUE','DAYS_LAST_DUE_1ST_VERSION',
               'DAYS_TERMINATION','NAME_TYPE_SUITE_prev_app',
               'WEEKDAY_APPR_PROCESS_START_prev_app','CREDIT_CURRENCY','AMT_REQ_CREDIT_BUREAU_HOUR',
                     'AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK',
                     'AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR',
                     'FLAG_LAST_APPL_PER_CONTRACT'],axis=1)

In [19]:
def modewithoutnan(column):
    return df[df[column]!=np.nan][column].mode()[0]


Below, I have identified columns which has a large amount of NaN values and filled it with the mean, mode, or 0 values. I selected means for continuous variables, and mode for disctrict variables. I placed 0.0 with columns that were looking for previous credit history. 

In [20]:
df = df.fillna(value={'DAYS_EMPLOYED':df['DAYS_EMPLOYED'][df['DAYS_EMPLOYED']!=365243].mean(),
                'OBS_30_CNT_SOCIAL_CIRCLE':modewithoutnan('OBS_30_CNT_SOCIAL_CIRCLE'),
                'DEF_30_CNT_SOCIAL_CIRCLE':modewithoutnan('DEF_30_CNT_SOCIAL_CIRCLE'),
                'OBS_60_CNT_SOCIAL_CIRCLE': modewithoutnan('OBS_60_CNT_SOCIAL_CIRCLE'),
                'DEF_60_CNT_SOCIAL_CIRCLE': modewithoutnan('DEF_60_CNT_SOCIAL_CIRCLE'),
                'AMT_ANNUITY_prev_app':df['AMT_ANNUITY_prev_app'].mean(),
                'AMT_APPLICATION':df['AMT_APPLICATION'].mean(),
                'AMT_DOWN_PAYMENT':df['AMT_DOWN_PAYMENT'].mean(),
                'AMT_GOODS_PRICE_prev_app':df['AMT_GOODS_PRICE_prev_app'].mean(),
                'OWN_CAR_AGE':df['OWN_CAR_AGE'].mean(),
                'CHANNEL_TYPE':modewithoutnan('CHANNEL_TYPE'),
                 'CNT_PAYMENT':modewithoutnan('CNT_PAYMENT'),
                'CODE_REJECT_REASON':modewithoutnan('CODE_REJECT_REASON'),
                'DAYS_DECISION':round(df['DAYS_DECISION'].mean(),0),
                'HOUR_APPR_PROCESS_START_prev_app':df['HOUR_APPR_PROCESS_START_prev_app'].mean(),
                'NAME_CASH_LOAN_PURPOSE':modewithoutnan('NAME_CASH_LOAN_PURPOSE'),
                'NAME_CONTRACT_STATUS':df['NAME_CONTRACT_STATUS'].mean(),
                'NAME_CONTRACT_TYPE_prev_app':modewithoutnan('NAME_CONTRACT_TYPE_prev_app'),
                'NAME_GOODS_CATEGORY':modewithoutnan('NAME_GOODS_CATEGORY'),
                'NAME_PAYMENT_TYPE':modewithoutnan('NAME_PAYMENT_TYPE'),
                'NAME_PORTFOLIO':modewithoutnan('NAME_PORTFOLIO'),
                'NAME_PRODUCT_TYPE':modewithoutnan('NAME_PRODUCT_TYPE'),
                'NAME_SELLER_INDUSTRY':modewithoutnan('NAME_SELLER_INDUSTRY'),
                'NAME_YIELD_GROUP':modewithoutnan('NAME_YIELD_GROUP'),
                'NFLAG_INSURED_ON_APPROVAL':df['NFLAG_INSURED_ON_APPROVAL'].mean(),
                'NFLAG_LAST_APPL_IN_DAY':modewithoutnan('NFLAG_LAST_APPL_IN_DAY'),
                'PRODUCT_COMBINATION':modewithoutnan('PRODUCT_COMBINATION'),
                'RATE_DOWN_PAYMENT':df['RATE_DOWN_PAYMENT'].mean(),
                'RATE_INTEREST_PRIMARY':df['RATE_INTEREST_PRIMARY'].mean(),
                'RATE_INTEREST_PRIVILEGED':df['RATE_INTEREST_PRIVILEGED'].mean(),
                'SELLERPLACE_AREA':modewithoutnan('SELLERPLACE_AREA'),
                'SK_ID_PREV_curr2':0.0,'AMT_ANNUITY':df['AMT_ANNUITY'].mean(),
                'AMT_CREDIT_MAX_OVERDUE':round(df['AMT_CREDIT_MAX_OVERDUE'].mean(),0),
                'AMT_CREDIT_SUM':0.0,'AMT_CREDIT_SUM_DEBT':0,'AMT_CREDIT_SUM_LIMIT':0.0,
                'AMT_CREDIT_SUM_OVERDUE':0.0,'CNT_CREDIT_PROLONG':0.0,
                'CREDIT_ACTIVE':0.0,'CREDIT_DAY_OVERDUE':0.0,'CREDIT_TYPE':0.0,
                'DAYS_CREDIT':0.0,'DAYS_CREDIT_ENDDATE':0.0,'DAYS_CREDIT_UPDATE':0.0,
                'DAYS_ENDDATE_FACT':0.0,'SK_ID_BUREAU':0.0,'AMT_INSTALMENT':0.0,
                'AMT_PAYMENT':0.0,'DAYS_ENTRY_PAYMENT':0.0,'DAYS_INSTALMENT':0.0,
                'NUM_INSTALMENT_NUMBER':0.0,'NUM_INSTALMENT_VERSION':0.0,'SK_ID_PREV_instpay':0.0,
                'AMT_INSTAL_PAY_DIFF':0.0,'NUM_INSTAL_VERSION_NUM_DIFF':0.0,
                'DAY_INSTAL_ENTRY_DIFF':0.0,'AMT_CREDIT_prev_app':0.0,'NAME_CLIENT_TYPE':0.0,
                     'OCCUPATION_TYPE':'Other','AMT_GOODS_PRICE_app_train':df['AMT_GOODS_PRICE_app_train'].mean()})

In [21]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 100002 to 216090
Columns: 110 entries, TARGET to BANKED
dtypes: bool(1), float64(66), int64(33), object(10)
memory usage: 84.0+ MB


In [23]:
df = df.dropna()

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99991 entries, 100002 to 216090
Columns: 110 entries, TARGET to BANKED
dtypes: bool(1), float64(66), int64(33), object(10)
memory usage: 84.0+ MB


In [24]:
df.to_pickle('df_pickle.pickle')